# Topic Classification

In this notebook we use the output from the topic model and our immersion journal/manual to subset a training set of tweets about the Australian Bushfires. 

In [1]:
#Importing relevant packages
import numpy as np 
import pandas as pd
from tqdm import tqdm #to create a progress bar
#Machine learning packages
from sklearn.linear_model import LogisticRegression
#Packages to create DFM
from sklearn.feature_extraction.text import CountVectorizer
#Packages for visualization
import matplotlib.pyplot as plt
import seaborn as sns
#Word embeddings
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors
#Gary King et. al key-words
from keyword_algorithm import *
#Remove unwarranted warnings
pd.options.mode.chained_assignment = None  # default='warn'

## Implementation of Gary Kings semi-automated keyword retrival

In [2]:
#Load and prepare data
data = pd.read_csv("data/final_df.csv", index_col=0)
data = data.dropna(subset = ["lemmas"]).reset_index(drop = True)
data["index_col"] = data.index
data.to_csv("data/query_df.csv")

In [81]:
#Download model from https://fasttext.cc/docs/en/english-vectors.html
model_dir = "data/wiki-news-300d-1M.vec"
fasttext = KeyedVectors.load_word2vec_format(model_dir)

In [158]:
class QueryBuilder:
    
    def __init__(self, emb_model):
        
        self.query = Keywords()
        #Load the data. Change path if necessary
        path = 'data/query_df.csv'
        self.query.LoadDataset(path, text_colname='lemmas', 
                    date_colname="created_at", id_colname="index_col")
        #Load Gensim word embeddings model 
        self.we = emb_model
            
    def get_keywords(self, its = 2, top_n = 10, refkeys = [], tarkeys = [], algorithms = ['nbayes', 'logit']), 
                     date_start = "2019-06-01", date_end = "2020-05-30":
        """
        Loops over King. et. al algorithm to extract relevant keywords used for building a 
        boolean query to subset relevant Tweets in the dataset.
        ---------
        arguments:
            - its: Iterations to run the algorithm
            - top_n: integer of how many of the most predictive keywords to extract in each iteration
            - refkeys: list of initial keywords used to create reference set of tweets
            - tarkeys: list of initial keywords used to limit the search set
            - algorithms: list of classifiers to run for extracting keywords 
            - date_start: y/m/d of start date for relevant tweets
            - date_end: y/m/d of end date for relevant tweets
        -----------    
        returns:
            - list of accepted keywords
            - list of rejected keywords
            - list of nontarget keywords
        """
        
        accepted_keywords = []
        rejected_keywords = []
        nontarget_keywords = []
        
        #Begin loop for mining search set
        for it in range(its):
            print("-"*66)
            print(f"STARTING ITERATION: {it}!")
            if it == 0:
                print("INITIAL REFERENCE KEYS: {refkeys} \n INITIAL TARGET KEYS: {tarkeys}")
            print("-"*66)
            
            #Build reference set of tweets
            self.query.ReferenceSet(any_words=refkeys, date_start=date_start, date_end=date_end)
            
            #Use accepted keys as search keys if not the first iteration
            if it > 0:
                #fit model on searchset and find target
                self.query.SearchSet(any_words= accepted_keywords, date_start=date_start, date_end=date_end)
            #Use accepted keys to demarcate the search set
            else:
                self.query.SearchSet(any_words = tarkeys, date_start=date_start, date_end=date_end)
            
            
            #Run King algorithm to find keywords.
            self.query.ProcessData(stem = False, keep_twitter_symbols=False,
                                   remove_wordlist=refkeys)
            self.query.ReferenceKeywords()
            self.query.ClassifyDocs(min_df=5, ref_trainprop=1, algorithms=algorithms)
            self.query.FindTargetSet()
            self.query.FindKeywords()
            
            #Extract target keywords from algorithm results
            target_keywords = self.query.target_keywords[:top_n]
            #Also get the reference set keywords to loop over
            target_keywords += self.query.reference_keywords[:top_n]
            #Append unique nontarget keywords to list of nontarget keys
            for nonkey in self.query.nontarget_keywords[:100]:
                if nonkey not in nontarget_keywords:
                    nontarget_keywords.append(nonkey)
            
            #Loop over each relevant keyword from reference and found target keywords
            for keyword in target_keywords:
                #Check if keyword has already been rejected or accepted
                if keyword in accepted_keywords or keyword in rejected_keywords:
                    continue
                else:
                    inp = input(f"Keep {keyword.upper()} yes or no?")
                    if inp == "yes":
                        accepted_keywords.append(keyword)
                        #get similar keywords through most similar pretrained embeddings
                        inp2 = input(f"Look at {keyword.upper()}'s most similar word embeddings, yes or no?")
                        if inp2 == "yes":
                            #Look if keyword exist in embedding model dictionary
                            try:
                                embeddings = [emb[0] for emb in self.we.most_similar(keyword)]
                                for emb in embeddings:
                                    if emb.lower() in accepted_keywords or emb.lower() in rejected_keywords:
                                        continue
                                    else:
                                        inp3 = input(f"Keep embedding {emb.upper()} yes or no?")
                                        if inp3 == "yes":
                                            accepted_keywords.append(emb)
                                        elif inp3 =="no":
                                            rejected_keywords.append(emb)
                            except:
                                print(f"{keyword.upper()} embedding not present in Model!")
                                pass
                        elif inp2 == "no":
                            pass
                    elif inp == "no":
                        rejected_keywords.append(keyword)
                        
            #Add custom keyword(s) in the end of the loop. Either as list or single keyword
            inp4 = input(f"Do you wish to add any further keywords? If yes, Type keyword: ")
            if inp4:
                if isinstance(inp4, list):
                    [accepted_keywords.append(key) for key in inp4]
                else:
                     accepted_keywords.append(inp4)
            else:
                pass
            
            print("-"*66)
            print(" "*20, f"CURRENT KEYWORDS AFTER ITTERATION {it}")
            print("-"*66)
            print(f"ACCEPTED: \n {accepted_keywords}")
            print(f"REJECTED: \n {rejected_keywords}")
            
        
        keywords = {"accepted_keys":accepted_keywords, "rejected_keys":rejected_keywords,"nontarget_keys":nontarget_keywords}
        
        return keywords
        
        
        

In [339]:
query = QueryBuilder(fasttext)

Keyword object initialized.
Loaded corpus of size 148540 in 3.0 seconds.


In [ ]:
keywords = query.get_keywords(2, top_n=10, refkeys=["#bushfire", "#bushfires", "bushfire", "bushfires"], tarkeys = ["fire"])

In [250]:
query = "|".join(keywords["accepted_keys"])

## Mlogit Model - Average Marginal Effect Estimation 

Here we use the key-words from Kings model to subset tweets. Next we use this subset to predict the party of the tweet and analyze the coefficients

In [162]:
class MlogitMargins:
    """
    Calculates marginal effect of multinomial logit coefficients with bootstraped confidence interval.
    See https://github.com/alicehwu/gendered_language/blob/master/gendered_language_2018.pdf for a 
    reference on a similar approach but with binary classification.
    """
    def __init__(self, X, y):
        #Define data
        self.vect = CountVectorizer(ngram_range=(1,2))
        self.X = self.vect.fit_transform(X)
        self.y = y
        print("Fitting model and calculating margins...")
        #Fit model and calculate average marginal effects
        self.margins, self.fitted_model = self.avg_margins(self.X, self.y)
        

    def avg_margins(self, X, y):
        """
        Calculates average marginal effect of coefficients in multinomial logit model, where each coefficient is 
        a word token and is calculated as:
        avg_margin_jk = beta_jk * 1/N * sum{P(y_i = J) * 1-P(y_i = J)} for each token k and class j.
        See: https://math.stackexchange.com/questions/863258/deriving-marginal-effects-in-multinomial-logit-model
        -----------
        Returns:
            - DataFrame with average margins of shape classes_j * tokens_k
            - The fitted mlogit model 
        """
        #Fit the model
        model = self.fit_model(X, y)
        #Get probabilities for each obs i belonging to class j. shape = j * N
        probas = model.predict_proba(X)
        #Get coefficients. Shape j_classes * k_coefficients
        betas = model.coef_

        margins = {}
        #Loop over each class 
        for class_j in range(len(model.classes_)):
            #Extract corresponging betas. shape 1*k
            for beta_jk in betas[class_j]:
                
                #Calculate avg margins for the jth class and kth beta (token)
                margins_jk = beta_jk * 1/probas.shape[0] * (probas[:,class_j] * (1 - probas[:,class_j])).sum()
                
                #Append avg margins to dictionary
                if model.classes_[class_j] not in margins.keys():
                    margins[model.classes_[class_j]] = [margins_jk]
                else:
                    margins[model.classes_[class_j]].append(margins_jk)
        
        #Extract the token name corresponding to the avg margins   
        margins["token"] = self.vect.get_feature_names()
        margins_df = pd.DataFrame(margins)
        
        return margins_df, model
    
    def fit_model(self, X, y, max_iter = 10000, penalty = "none", class_weight = "balanced",
                  verbose = False, fit_intercept = True, multi_class = "multinomial"):
        
        """
        Fits sklearn multinomial logistic model on data. Default penalty set to "none" for unbaised estimators.
        """

        mlogit = LogisticRegression(random_state=42, penalty=penalty, solver="saga", 
                                    max_iter = max_iter, class_weight = class_weight,
                                    fit_intercept=fit_intercept, multi_class=multi_class,
                                    verbose=verbose).fit(X, y) #   

        return mlogit
    
    def bootstrap_ci(self, alpha = 0.05, n_samples = 500, sample_prop = 0.4):
        """
        Uses bootstraping to calculate confidence interval around average marginal effects estimate.
        -------------
        Arguments:
            - alpha: deterimines confidence level of the interval
            - n_samples: amount of bootstrap samples
            - sample_prop: bootstramp sample size as proportion of original sample
        -------------
        Return:
            - Pandas dataframe with confidence interval for average marginal effect
              of each coefficient.
        """
        statistics = [] #List for bootstrap results
        
        #Define the bootstrap sample size based on proportion of total
        n_size = int(self.X.shape[0] * sample_prop)
        print(f"Number of obs in bootstrap samples {n_size}...")
        start = time.time()
        for i in tqdm(range(n_samples)):

            #Draw random sample from X and y with replacement
            idx = np.random.choice(np.arange(self.X.shape[0]), n_size, replace=True)
            X_sample = self.X[idx]
            y_sample = self.y[idx]
            #Calculate marginal effect for sample
            margins = self.avg_margins(X_sample, y_sample)[0]
            statistics.append(margins)
        
        #Join the resulting dataframes of margins
        statistics = pd.concat(statistics)
        print(f"Boostraping Done. Calculating {1-alpha}% confidence interval...")
        #From bootstrap results get lower and upper CI limits based on alpha
        statistics = statistics.groupby("token").quantile([alpha, 1-alpha]).reset_index()        
        print("Time to complete: ", time.time() - start)

        return statistics
    

In [140]:
#NOTE: This data is just for testing - not final subset.
mlogit_data = data.loc[(data["lemmas"].str.contains("bushfire|bushfires|#bushfire|#bushfires|disaster|kaoala|fire|flames|firemen") == True) & 
                       (data["lemmas"].str.contains("corona|covid") == False) & 
                       (data["created_at"] >= "2019-06-01") & 
                       (data["created_at"] <= "2020-06-01")]

#Reduce classes to predict. Otherwise bad model performance.
mlogit_data = mlogit_data.loc[mlogit_data["party"].isin(["Australian Greens",
                                                         "Australian Labor Party", 
                                                         "Liberal Party of Australia",
                                                          ])].dropna(subset = ["lemmas"]).reset_index(drop = True)

In [163]:
X, y = mlogit_data["lemmas"], mlogit_data["party"]
mlogit = MlogitMargins(X, y)

Fitting model and calculating margins...


In [167]:
mlogit.margins.sort_values("Australian Greens", ascending=False).head(15)

,Australian Greens,Australian Labor Party,Liberal Party of Australia,token
4095,0.003002,-0.002033,-0.001520,climate
4155,0.002635,-0.002593,-0.000964,climateemergency
4222,0.002226,-0.001706,-0.001036,coal
7119,0.001402,-0.001060,-0.000659,especially case
3532,0.001402,-0.001060,-0.000659,case bushfire
13008,0.001315,-0.000705,-0.000751,many
5449,0.001281,-0.000797,-0.000681,cut
1850,0.001191,-0.000186,-0.000888,bad
4111,0.001151,-0.001591,-0.000211,climate emergency
3530,0.001125,-0.000400,-0.000736,case


In [155]:
mlogit_ci = mlogit.bootstrap_ci(alpha=0.05, n_samples=25)

  0%|          | 0/25 [00:00<?, ?it/s]

Number of obs in bootstrap samples 880...


100%|██████████| 25/25 [01:10<00:00,  2.80s/it]


Boostraping Done. Calculating 0.95% confidence interval...
Time to calculate:  0.403839111328125


In [157]:
mlogit_ci.head(10)

,token,level_1,Australian Greens,Australian Labor Party,Liberal Party of Australia
0,abandon,0.05,-0.000637,0.000031,-0.001071
1,abandon,0.95,-0.000007,0.001977,-0.000017
2,abandon morrison,0.05,-0.000022,0.000000,-0.000259
3,abandon morrison,0.95,0.000000,0.000866,0.000000
4,abate,0.05,-0.000114,0.000000,-0.000150
5,abate,0.95,0.000000,0.000387,0.000000
6,abate community,0.05,-0.000114,0.000000,-0.000150
7,abate community,0.95,0.000000,0.000387,0.000000
8,abc,0.05,-0.000932,-0.000534,-0.001135
9,abc,0.95,-0.000061,0.002493,0.000369
